To connect to an Azure workspace replace the following variables with your own values.

In [1]:
subscription_id = 'xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx'
resource_group = 'myresourcegroup'
workspace_name = 'myworkspace'
location = 'westus'

Import the Q# package. Set the target profile to be the base profile.

In [2]:
import qsharp

qsharp.init(target_profile=qsharp.TargetProfile.Base)

The following will generate a compiler error because it uses a feature unsupported by the base profile.

In [3]:
%%qsharp

operation NotAllowed() : Unit {
    use q = Qubit();
    let result = M(q);
    if (result == Zero) {
        Message("The result is Zero");
    }
    Reset(q);
}

Qsc.BaseProfCk.ResultComparison

  × cannot compare measurement results
   ╭─[4:1]
 4 │     let result = M(q);
 5 │     if (result == Zero) {
   ·         ──────────────
 6 │         Message("The result is Zero");
   ╰────
  help: comparing measurement results is not supported when performing base
        profile QIR generation
Qsc.BaseProfCk.ResultLiteral

  × result literals are not supported
   ╭─[4:1]
 4 │     let result = M(q);
 5 │     if (result == Zero) {
   ·                   ────
 6 │         Message("The result is Zero");
   ╰────
  help: result literals `One` and `Zero` are not supported when performing
        base profile QIR generation


Define some more Q# operations. This should compile without error.

In [4]:
%%qsharp

operation Random() : Result {
    use q = Qubit();
    H(q);
    let result = M(q);
    Reset(q);
    return result
}

operation RandomNBits(N: Int): Result[] {
    mutable results = [];
    for i in 0 .. N - 1 {
        let r = Random();
        set results += [r];
    }
    return results
}

Submit to Azure Quantum with a custom entry expression.

Make sure the `azure-quantum` package is installed.

In [ ]:
%pip install azure-quantum

In [ ]:
import azure.quantum

operation = qsharp.compile("RandomNBits(4)")

workspace = azure.quantum.Workspace(
    subscription_id=subscription_id,
    resource_group=resource_group,
    name=workspace_name,
    location=location,
)
target = workspace.get_targets("rigetti.sim.qvm")
job = target.submit(operation, "my-azure-quantum-job", input_params={ "count": 100 })
job.get_results()